### Imports

In [2]:
from pyod.models.ecod import ECOD
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 689 µs (started: 2023-06-08 19:24:57 -05:00)


### Parameters

In [3]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = params["method"]
print("Method:\t\t", method)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

option = params["option"]
print("Option:\t\t", option)

if option:
    
    subgroups_id_op = {}
    for group in groups_id:
        subgroups_id_op[group] = [option]
else:
    subgroups_id_op = subgroups_id
print("Subgroups id op:", subgroups_id_op)



Exp:		 exp5
Method:		 dgi
Dimension:	 3
Groups id:	 ['WT', 'zwf1^', 'pck1^']
Subgroups id:	 {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
Option:		 
Subgroups id op: {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
time: 2.58 ms (started: 2023-06-08 19:24:58 -05:00)


### Edge embeddings

In [4]:
# get edges embeddings

edge_embeddings_global(exp, method, groups_id, subgroups_id_op)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, groups_id[0], subgroups_id_op[groups_id[0]][0]), index_col=[0, 1])
df_edge_embeddings.head()

0             1         2
0 1  0.000623  1.468499e-07  0.001172
  2  0.000002  7.043716e-10  0.000004
  3  0.000964  1.503268e-07  0.001826
  4  0.000131  1.577536e-10  0.000256
  5  0.000228  6.944068e-08  0.000426

time: 4.9 s (started: 2023-06-08 16:53:32 -05:00)


### Concat edge embeddings

In [ ]:
for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.DataFrame()
    k = 0
    for subgroup in tqdm(subgroups_id_op[group]):
        df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, group, subgroup), index_col=[0, 1])
        df_edge_embeddings["subgroup"] = [k] * len(df_edge_embeddings)
        df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])
        k += 1
    
    df_edge_embeddings_concat.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index=True)

100%|██████████| 3/3 [03:33<00:00, 71.20s/it]

time: 3min 33s (started: 2023-06-08 16:53:51 -05:00)


In [ ]:
df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, groups_id[0], option), index_col=[0, 1])
df_edge_embeddings_concat.head()

0             1         2  subgroup
0 1  0.000623  1.468499e-07  0.001172         0
  2  0.000002  7.043716e-10  0.000004         0
  3  0.000964  1.503268e-07  0.001826         0
  4  0.000131  1.577536e-10  0.000256         0
  5  0.000228  6.944068e-08  0.000426         0

time: 5.11 s (started: 2023-06-08 16:57:34 -05:00)


In [ ]:
# plot edge embeddings concat

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    x = df_edge_embeddings_concat.iloc[:, 0]
    y = df_edge_embeddings_concat.iloc[:, 1]
    z = df_edge_embeddings_concat.iloc[:, 2]

    # Creating figure
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")

    # Creating plot
    ax.scatter3D(x, y, z, c=df_edge_embeddings_concat.iloc[:, -1], alpha=0.1)
    # plt.title("Dimension: {}".format(dimension))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_concat_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.close()

  0%|          | 0/3 [05:06<?, ?it/s]

time: 5min 6s (started: 2023-06-08 16:58:15 -05:00)


### Outliers detection

In [ ]:
# Outlier detection (HDBSCAN)

""" df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

X_train = df_edge_embeddings_concat.iloc[:, :-1]
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)

threshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)
outliers = np.where(clusterer.outlier_scores_ > threshold)[0]
print(len(outliers))
outliers

inliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)
print(len(inliers))
inliers """

' df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])\n\nX_train = df_edge_embeddings_concat.iloc[:, :-1]\nclusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)\n\nthreshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)\noutliers = np.where(clusterer.outlier_scores_ > threshold)[0]\nprint(len(outliers))\noutliers\n\ninliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)\nprint(len(inliers))\ninliers '

time: 3.71 ms (started: 2023-06-08 14:03:09 -05:00)


In [ ]:
# outlier detection (ECOD)

dict_df_edge_embeddings_concat_outlier = {}
dict_df_edge_embeddings_concat_filter = {}

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    X_train = df_edge_embeddings_concat.iloc[:, :-1]

    clf = ECOD()
    clf.fit(X_train)

    X_train["labels"] = clf.labels_ # binary labels (0: inliers, 1: outliers)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat.copy()
    df_edge_embeddings_concat_filter["labels"] = clf.labels_
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["labels"] == 0]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, :-1]

    dict_df_edge_embeddings_concat_outlier[group] = X_train
    dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [01:03<00:00, 21.32s/it]

time: 1min 4s (started: 2023-06-08 17:43:52 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0             1         2  subgroup
0 1  0.000623  1.468499e-07  0.001172         0
  3  0.000964  1.503268e-07  0.001826         0
  5  0.000228  6.944068e-08  0.000426         0
  7  0.002065  4.559423e-07  0.003888         0
  8  0.003653  3.862995e-07  0.006963         0

time: 10.8 ms (started: 2023-06-08 17:45:51 -05:00)


In [ ]:
# plot outliers/inliers

for group in tqdm(groups_id):
    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")

    df_aux = dict_df_edge_embeddings_concat_outlier[group]
    print("Total:", len(df_aux))
    
    temp = df_aux[df_aux["labels"] == 0]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="red", alpha=0.005)
    print("Num. of inliers:", len(temp))

    temp = df_aux[df_aux["labels"] == 1]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="gray", alpha=0.005)
    print("Num. of inliers:", len(temp))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_outlier_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.close()

  0%|          | 0/3 [00:00<?, ?it/s]

Total: 12257101
Num. of inliers: 11031391
Num. of inliers: 1225710


  0%|          | 0/3 [04:48<?, ?it/s]

time: 4min 48s (started: 2023-06-08 17:05:26 -05:00)


###  Filter common edges

In [ ]:
print(subgroups_id)
print(subgroups_id_op)


{'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
{'WT': ['dyn'], 'zwf1^': ['dyn'], 'pck1^': ['dyn']}
time: 357 µs (started: 2023-06-08 17:45:58 -05:00)


In [ ]:
# mapping idx with id

for group in tqdm(groups_id):
    dict_df_nodes = {}
    for subgroup in subgroups_id_op[group]:
        df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup),
                               dtype={"id": "string"})
        dict_df_nodes[subgroup] = df_nodes
    
    # mapping
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
    list_index = []
    
    for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
        df_nodes = dict_df_nodes[subgroups_id_op[group][row[-1]]]
        list_index.append((df_nodes.iloc[row[0][0], -1], df_nodes.iloc[row[0][1], -1]))
    
    # set new index
    df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)

11031391it [05:13, 35225.81it/s]it/s]
15391298it [07:10, 35749.70it/s]4, 322.24s/it]
5445463it [02:35, 35084.19it/s]32, 392.89s/it]
100%|██████████| 3/3 [15:24<00:00, 308.27s/it]

time: 15min 24s (started: 2023-06-08 17:46:09 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0             1         2  subgroup
A59.0049 A274.0123  0.000623  1.468499e-07  0.001172         0
         A369.1213  0.000964  1.503268e-07  0.001826         0
         A503.0516  0.000228  6.944068e-08  0.000426         0
         A558.4273  0.002065  4.559423e-07  0.003888         0
         A641.1223  0.003653  3.862995e-07  0.006963         0

time: 9.22 ms (started: 2023-06-08 18:02:44 -05:00)


In [ ]:
# format id
if option:
    for group in tqdm(groups_id):
        # format
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        list_index = []

        for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
            list_index.append((row[0][0][1:], row[0][1][1:]))
        
        # set new index
        df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)
        df_edge_embeddings_concat_filter

11031391it [00:10, 1056642.42it/s]/s]
15391298it [00:14, 1067101.42it/s] 21.78s/it]
5445463it [00:04, 1129923.30it/s], 26.07s/it]
100%|██████████| 3/3 [01:02<00:00, 20.78s/it]

time: 1min 2s (started: 2023-06-08 18:02:52 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0             1         2  subgroup
59.0049 274.0123  0.000623  1.468499e-07  0.001172         0
        369.1213  0.000964  1.503268e-07  0.001826         0
        503.0516  0.000228  6.944068e-08  0.000426         0
        558.4273  0.002065  4.559423e-07  0.003888         0
        641.1223  0.003653  3.862995e-07  0.006963         0

time: 13.4 ms (started: 2023-06-08 18:04:08 -05:00)


In [ ]:
df_edge_embeddings_concat_filter.index[:5]

MultiIndex([('59.0049', '274.0123'),
            ('59.0049', '369.1213'),
            ('59.0049', '503.0516'),
            ('59.0049', '558.4273'),
            ('59.0049', '641.1223')],
           )

time: 3.4 ms (started: 2023-06-08 18:04:18 -05:00)


In [ ]:
# filter diferente edges

if option:
    dict_df_edge_embeddings_concat_filter_ = {}
    for group in tqdm(groups_id):
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter.index.get_level_values(0) != df_edge_embeddings_concat_filter.index.get_level_values(1)]
        dict_df_edge_embeddings_concat_filter_[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [00:02<00:00,  1.02it/s]

time: 2.96 s (started: 2023-06-08 18:04:27 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter_[groups_id[0]]
print(df_edge_embeddings_concat_filter.shape)
df_edge_embeddings_concat_filter.head()

(11008320, 4)


0             1         2  subgroup
59.0049 274.0123  0.000623  1.468499e-07  0.001172         0
        369.1213  0.000964  1.503268e-07  0.001826         0
        503.0516  0.000228  6.944068e-08  0.000426         0
        558.4273  0.002065  4.559423e-07  0.003888         0
        641.1223  0.003653  3.862995e-07  0.006963         0

time: 12.9 ms (started: 2023-06-08 18:04:33 -05:00)


In [ ]:
# count edges and filter by count

dict_df_edges_filter = {}
for group in tqdm(groups_id):
    # count
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]

    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[["level_0", "level_1"]].value_counts().to_frame()
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.columns = ["source", "target", "count"]

    # filter
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["count"] == len(subgroups_id[group])]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, [0, 1]]
    dict_df_edges_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [00:15<00:00,  5.27s/it]

time: 15.9 s (started: 2023-06-08 18:04:57 -05:00)


In [ ]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,307.0798,460.2078
1,244.1304,424.1145
2,244.1304,402.1765
3,244.1304,403.1034
4,244.1304,403.1935


time: 9.04 ms (started: 2023-06-08 18:05:20 -05:00)


In [ ]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28599 entries, 0 to 28598
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  28599 non-null  object
 1   target  28599 non-null  object
dtypes: object(2)
memory usage: 670.3+ KB
time: 14 ms (started: 2023-06-08 18:05:33 -05:00)


In [ ]:
# change data type
for group in tqdm(groups_id):
    df_edges_filter = dict_df_edges_filter[group]
    df_edges_filter[["source", "target"]] = df_edges_filter[["source", "target"]].astype("string")

100%|██████████| 3/3 [00:00<00:00, 22.30it/s]

time: 137 ms (started: 2023-06-08 18:05:38 -05:00)


In [ ]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,307.0798,460.2078
1,244.1304,424.1145
2,244.1304,402.1765
3,244.1304,403.1034
4,244.1304,403.1935


time: 4.37 ms (started: 2023-06-08 18:05:40 -05:00)


In [ ]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28599 entries, 0 to 28598
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  28599 non-null  string
 1   target  28599 non-null  string
dtypes: string(2)
memory usage: 670.3 KB
time: 12.7 ms (started: 2023-06-08 18:05:43 -05:00)


In [ ]:
# get weight by subgroups

dict_df_edges_filter_weight = get_weight_global(dict_df_edges_filter, exp, groups_id, subgroups_id)
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

100%|██████████| 3/3 [01:14<00:00, 24.68s/it]


,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
28138,109.0407,109.0514,0.799735,0.675500,0.820072,0.871161,0.809262
28598,109.0407,127.0513,0.791174,0.792589,0.782065,0.917810,0.870420
28534,109.0407,131.0462,0.802158,0.721020,0.613603,0.872517,0.867231
28417,109.0407,131.0824,0.808212,0.663848,0.719276,0.857543,0.913011
27300,109.0407,132.086,0.764665,0.614573,0.603960,0.893537,0.878882


time: 1min 14s (started: 2023-06-08 18:05:57 -05:00)


In [ ]:
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
28138,109.0407,109.0514,0.799735,0.675500,0.820072,0.871161,0.809262
28598,109.0407,127.0513,0.791174,0.792589,0.782065,0.917810,0.870420
28534,109.0407,131.0462,0.802158,0.721020,0.613603,0.872517,0.867231
28417,109.0407,131.0824,0.808212,0.663848,0.719276,0.857543,0.913011
27300,109.0407,132.086,0.764665,0.614573,0.603960,0.893537,0.878882


time: 13.7 ms (started: 2023-06-08 18:07:16 -05:00)


### Filter by STD and average weight

In [ ]:
def std_global(dict_df_edges_filter_weight, exp, method, groups_id, option, th=0.3, plot=True, save=False):
    dict_df_common_edges_std = {}

    for group in tqdm(groups_id):
        df_edges_filter_weight = dict_df_edges_filter_weight[group].copy()

        # calculate std
        df_edges_filter_weight["std"] = np.std(df_edges_filter_weight.iloc[:, 2:], axis=1)

        # filter std < 0.3
        df_edges_filter_weight_std = df_edges_filter_weight[df_edges_filter_weight["std"] < th]

        # average weight
        df_edges_filter_weight_std_avg = df_edges_filter_weight_std.iloc[:, :-1]
        df_edges_filter_weight_std_avg["weight"] = df_edges_filter_weight_std_avg.iloc[:, 2:].mean(axis=1)
        df_edges_filter_weight_std_avg = df_edges_filter_weight_std_avg.iloc[:, [0, 1, -1]]
        df_edges_filter_weight_std_avg.reset_index(drop=True, inplace=True)

        # save
        if save:
            df_edges_filter_weight_std_avg.to_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, group, option), index=False)
            
            G = nx.from_pandas_edgelist(df_edges_filter_weight_std_avg, "source", "target", edge_attr=["weight"])
            nx.write_gexf(G, "output/{}/common_edges/common_edges_{}_{}_{}.gexf".format(exp, method, group, option))

        # plot
        if plot:
            x = df_edges_filter_weight["std"]
            plt.hist(x, bins=100)
            plt.axvline(x=th, color="red", lw=1)
            l = len(df_edges_filter_weight) - len(df_edges_filter_weight_std)
            t = len(df_edges_filter_weight)
            plt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))
            plt.savefig("output/{}/plots/common_edges_std_{}_{}_{}.png".format(exp, method, group, option))
            # plt.show()
            plt.close()

        dict_df_common_edges_std[group] = df_edges_filter_weight_std_avg
    return dict_df_common_edges_std

time: 2.54 ms (started: 2023-06-08 18:09:38 -05:00)


In [ ]:
dict_df_common_edges = std_global(dict_df_edges_filter_weight, exp, method, groups_id, option, th=0.3, plot=True, save=True)
dict_df_common_edges[groups_id[0]].head()

100%|██████████| 3/3 [00:13<00:00,  4.61s/it]


,source,target,weight
0,109.0407,109.0514,0.795146
1,109.0407,127.0513,0.830812
2,109.0407,131.0462,0.775306
3,109.0407,131.0824,0.792378
4,109.0407,132.086,0.751123


time: 13.8 s (started: 2023-06-08 18:09:49 -05:00)


In [ ]:
df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, groups_id[0], option),
                              dtype={"source": "string", "target": "string"})
df_common_edges.head()

,source,target,weight
0,109.0407,109.0514,0.795146
1,109.0407,127.0513,0.830812
2,109.0407,131.0462,0.775306
3,109.0407,131.0824,0.792378
4,109.0407,132.086,0.751123


time: 20.7 ms (started: 2023-06-08 18:10:28 -05:00)


: 

In [ ]:
# show details

for group in tqdm(groups_id):
    df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, group, option))
    
    G = nx.from_pandas_edgelist(df_common_edges, "source", "target", edge_attr=["weight"])
    print("Group: {}".format(group))
    graph_detail(G)

 33%|███▎      | 1/3 [00:00<00:00,  2.14it/s]

Group: WT
Num. nodes: 1916
Num. edges: 23331



 67%|██████▋   | 2/3 [00:01<00:00,  1.29it/s]

Group: zwf1^
Num. nodes: 4367
Num. edges: 521944



100%|██████████| 3/3 [00:02<00:00,  1.30it/s]

Group: pck1^
Num. nodes: 5046
Num. edges: 454546

time: 2.31 s (started: 2023-06-06 16:14:23 -05:00)
